In [1]:
# General imports
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import collections
import os
import time
import re
import itertools

# Data Science
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Natural Language processing
import nltk

## Stemming
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize

# Learning Algorithms / estimators
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier

# Metrics
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# Feature Extraction
from sklearn.feature_extraction.text import TfidfVectorizer

# Process
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import StratifiedKFold
from sklearn.pipeline import Pipeline

# Plotting
from matplotlib.markers import MarkerStyle
from PIL import Image

# Corpus
from documentModel import DocumentModel as DM
from export_results import *
from utils import *

In [17]:
def save_image(image, url='../images/', name = 'default'):
    image.savefig(url + name)
    Image.open(url + name + '.png').convert('L').save(url + name + '.png')
    
def plot_image(x, y, title="title", ylim = [0, 1.02], xlim = [2, 50.5], 
               colors="rgbmyc", models=None, name="name", labels=[], ylabel = "ylabel", 
               loc="better", markers=".,ov<>", pattern=""):
    plt.figure(figsize=(14,13))
    plt.ylim(ylim)
    plt.xlim(xlim)
    plt.xlabel("Misclassification Cost Ratio")
    plt.ylabel(ylabel)
    plt.style.use('paper.mplstyle')
    
    filled_markers = ('<', 'D', 'o', '|', 'v', '>', 'p', 'd') #' '^', ', '>', '8', 's', 'p', '*', 'h', 'H', , 'd')
    fillstyles = ('full', 'full', 'full', 'full', 'top', 'none')

    i = 0
    for model in models:
        name = model[0]
        if re.match(pattern, name) is not None:
            plt.plot(x, y[name])
            marker = MarkerStyle(marker=filled_markers[i], fillstyle=fillstyles[i])
            plt.scatter(x, y[name], marker=marker, s=300, label=model[0])
            i+=1
    
    plt.xticks(x, labels, rotation='vertical')
    plt.legend(loc=loc, prop={'size':30})

    save_image(plt,'../images/', name)
    plt.show()
    
def filter_results(recalls, precisions, models=None, pattern=".*"):

    recalls_avg = {}
    precisions_avg = {}

    for model in models:
        name = model[0]
        if re.match(pattern, name) is not None:
            recalls_avg[name] = recalls[name]
            precisions_avg[name] = precisions[name]
    
    return recalls_avg, precisions_avg
    
def normalize(recall_avg, precision_avg):
    recall_avg_normalized = {}

    for key, value in recall_avg.items():
        recall_avg_normalized[key] = []

    for key, values in recall_avg.items():
        for value in values:
            recall_avg_normalized[key].append(float(value * 100))
 

    precision_avg_normalized = {}

    for key, value in precision_avg.items():
        precision_avg_normalized[key] = []

    for key, values in precision_avg.items():
        for value in values:
            precision_avg_normalized[key].append(float(value * 100)) 
            
    return recall_avg_normalized, precision_avg_normalized

def set_baseline(name):
    recall_baseline = recall_avg_normalized[name]
    precision_baseline = precision_avg_normalized[name]
    
    return recall_baseline, precision_baseline

#".*Stemming.*(Bayes|SVM)"    
def summary(recalls, precisions, pattern=".*"):
    r, p = filter_results(recalls, precisions, pipes, pattern) 
    max_recall = 0.0
    best_precision = 0.0
    best_cost = "1:1"
    best_algorithm = ''

    for (name, recalls), (name_2, precisions) in zip(r.items(), p.items()):
        print("Executing: " + name + " ...")
        recs = []
        pres = []
        recs.append(np.mean(recalls))
        pres.append(np.mean(precisions))
        for i, (recall, precision) in enumerate(zip(recalls, precisions)):
            if recall > max_recall:
                best_cost = labels[i]
                max_recall = recall
                best_precision = precisions[i]
                best_algorithm = name
            if recall == max_recall and precision > best_precision:
                best_cost = labels[i]
                max_recall = recall
                best_precision = precisions[i]
                best_algorithm = name
                
    return best_cost, max_recall, best_precision, best_algorithm, np.mean(recs), np.mean(pres) 

In [2]:
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = SnowballStemmer(language="spanish")
    
    def __call__(self, doc):
        return [self.wnl.stem(t) for t in word_tokenize(doc)]

In [3]:
def init():
    recall_avg = {}
    recall_std = {}
    precision_avg = {}
    precision_std = {}
    pipelines = []

    for element in itertools.product(estimators, nlp):
        name = element[1][0] + '_' + element[0][0]
        pipeline = Pipeline([('nlp', element[1][1]), ('clf', element[0][1])])
        pipelines.append((name , pipeline))
        recall_avg[name] = []
        precision_avg[name] = []
        recall_std[name] = []
        precision_std[name] = []
        
    return pipelines, recall_avg, recall_std , precision_avg, precision_std

# Business Rule Classification

In [4]:
stop_words = ['a', 'bajo', 'en', 'para','un', 'la', 'el', 'los', 'las', 'su', 'sus', 'través', 'al','con', \
             'más', 'muy', 'cual', 'poco', 'que']

print("Transforming annotated files into training datasets...")
dm = DM()
fito_dataset = dm.get_sentences(0)
X, y = fito_dataset["data"], fito_dataset["target"]

print("OK")

Transforming annotated files into training datasets...
OK


## NLP Exploration

In [5]:
rule_data = pd.DataFrame({"rule": X, "tag": y})
permissions = rule_data[rule_data["tag"] == 0]
prohibitions = rule_data[rule_data["tag"] == 1]

In [6]:
permissions.head()

,rule,tag
0,Aplicar sobre el suelo en pequeños montones o ...,0
3,"Aplicar en pulverización normal, variando la d...",0
4,En la etiqueta deberán figurar las instruccion...,0
9,"Aplicar en pulverización normal, dependiendo d...",0
14,Aplicar en pulverización normal,0


In [7]:
prohibitions.head()

,rule,tag
1,Evitar que el producto caiga sobre las plantas...,1
2,Advertir en la etiqueta que es peligroso para ...,1
5,No mezclar con aceites ni productos de reacció...,1
6,No aplicar aceites minerales durante los 21 dí...,1
7,No aplicar en cultivos cuyos frutos sean desti...,1


## Experiment Variables

In [8]:
labels = ["1:1","1:2","1:4","1:6","1:10","1:25", "1:50", r"1:$10^2$", r"1:$10^3$", r"1:$10^6$"]
# like the ShuffleSplit strategy, stratified random splits do not guarantee 
#that all folds will be different, although this is still very likely for sizeable datasets
sss = StratifiedKFold(y, n_folds = 10, random_state = 2016)
costs = np.array([2, 3, 4, 6, 10, 25, 50, 100, 1000, 1000000])
axis_costs = np.arange(1,11,1)
cxlim = [0.8, 10.15]
estimators = [("Naive Bayes", MultinomialNB(fit_prior=False)), 
              ("Random Forest", RandomForestClassifier(n_estimators=20)), 
              ("SVM", SVC(kernel='linear', C = 0.1)),
              ("Baseline", DummyClassifier(strategy = "constant", constant=1))]

nlp = [("None", TfidfVectorizer(use_idf = True, stop_words=stop_words)),
       ("Stemming", TfidfVectorizer(use_idf = True, stop_words=stop_words, tokenizer=LemmaTokenizer())), 
       ("Bigrams", TfidfVectorizer(use_idf = True, stop_words=stop_words, ngram_range=(2, 2))), 
       ("Combination", TfidfVectorizer(use_idf = True, stop_words=stop_words, ngram_range=(1, 2)))]

In [12]:
pipes, r_a, r_s, p_a, p_s = init()

## Main Experiment Loop

In [9]:
def experiment(pattern = ".*", to_normalize = True):

    t0 = time.time()
    pipes, recall_avg, recall_std, precision_avg, precision_std = init()

    for estimator in pipes:
        name = estimator[0]
        if re.match(pattern, name) is not None:
            print("Trying: " + name + " ...")
            for cost in costs:
                extractor = estimator[1].steps[0][1].fit(X)
                if "Naive Bayes" in name:
                    model = estimator[1].set_params(clf__class_prior=[1/cost, (cost-1)/cost]).steps[1][1]
                elif "Random Forest" in name:
                    model = estimator[1].set_params(clf__class_weight={1:cost-1}).steps[1][1]
                elif "SVM" in name:
                    model = estimator[1].set_params(clf__class_weight={1:cost-1}).steps[1][1]
                elif "Baseline" in name:
                    model = estimator[1].steps[1][1]
            
                precisions = []
                recalls = []
                for train_index, test_index in sss:
                    X_train, X_test = \
                        extractor.transform(X)[train_index], extractor.transform(X)[test_index]
                    y_train, y_test = y[train_index], y[test_index]
                    model.fit(X_train, y_train)
                    precisions.append(precision_score(y_test, model.predict(X_test), pos_label=1))
                    recalls.append(recall_score(y_test, model.predict(X_test), pos_label=1))
                recall_avg[name].append(np.average(recalls))
                recall_std[name].append(np.std(recalls))
                precision_avg[name].append(np.average(precisions))
                precision_std[name].append(np.std(precisions))
                
    t1 = time.time()
    print()
    print("Execution time: %.3f min" % ((t1 - t0)/60))
    
    if to_normalize == True:
        recall_avg_normalized, precision_avg_normalized = normalize(recall_avg, precision_avg)
        return recall_avg_normalized, precision_avg_normalized
    
    return recall_avg, precision_avg

In [10]:
recall_avg_normalized, precision_avg_normalized = experiment()        

Trying: None_Naive Bayes ...
Trying: Stemming_Naive Bayes ...
Trying: Bigrams_Naive Bayes ...
Trying: Combination_Naive Bayes ...
Trying: None_Random Forest ...
Trying: Stemming_Random Forest ...
Trying: Bigrams_Random Forest ...
Trying: Combination_Random Forest ...
Trying: None_SVM ...
Trying: Stemming_SVM ...
Trying: Bigrams_SVM ...
Trying: Combination_SVM ...
Trying: None_Baseline ...
Trying: Stemming_Baseline ...
Trying: Bigrams_Baseline ...
Trying: Combination_Baseline ...

Execution time: 10.084 min


## All classifiers

In [157]:
best_cost, max_recall, best_precision, best_algorithm, mean_recall, mean_precision = \
    summary(recall_avg_normalized, precision_avg_normalized, pattern=".*")

Executing: Combination_Baseline ...
Executing: Stemming_Naive Bayes ...
Executing: None_Naive Bayes ...
Executing: None_SVM ...
Executing: Stemming_Random Forest ...
Executing: Combination_Naive Bayes ...
Executing: Stemming_SVM ...
Executing: Bigrams_Naive Bayes ...
Executing: None_Baseline ...
Executing: None_Random Forest ...
Executing: Bigrams_SVM ...
Executing: Combination_Random Forest ...
Executing: Bigrams_Random Forest ...
Executing: Combination_SVM ...
Executing: Stemming_Baseline ...
Executing: Bigrams_Baseline ...


In [158]:
print(best_cost)
print(max_recall)
print(best_precision)
print(best_algorithm)
print(mean_recall)
print(mean_precision)

1:10
100.0
44.718557180116036
Bigrams_Naive Bayes
100.0
34.4986425339


## Bayes

In [165]:
best_cost, max_recall, best_precision, best_algorithm, mean_recall, mean_precision = \
    summary(recall_avg_normalized, precision_avg_normalized, pattern=".*Bayes")

Executing: Combination_Naive Bayes ...
Executing: Stemming_Naive Bayes ...
Executing: None_Naive Bayes ...
Executing: Bigrams_Naive Bayes ...


In [166]:
print(best_cost)
print(max_recall)
print(best_precision)
print(best_algorithm)
print(mean_recall)
print(mean_precision)

1:10
100.0
44.718557180116036
Bigrams_Naive Bayes
99.362745098
45.8460064913


## SVM

In [167]:
best_cost, max_recall, best_precision, best_algorithm, mean_recall, mean_precision = \
    summary(recall_avg_normalized, precision_avg_normalized, pattern=".*SVM")

Executing: Stemming_SVM ...
Executing: Combination_SVM ...
Executing: Bigrams_SVM ...
Executing: None_SVM ...


In [169]:
print(best_cost)
print(max_recall)
print(best_precision)
print(best_algorithm)
print(mean_recall)
print(mean_precision)

1:6
99.41176470588235
51.59298770001042
Bigrams_SVM
92.2124183007
68.7617875467


## Random Forest

In [170]:
best_cost, max_recall, best_precision, best_algorithm, mean_recall, mean_precision = \
    summary(recall_avg_normalized, precision_avg_normalized, pattern=".*Random.*")

Executing: None_Random Forest ...
Executing: Combination_Random Forest ...
Executing: Bigrams_Random Forest ...
Executing: Stemming_Random Forest ...


In [171]:
print(best_cost)
print(max_recall)
print(best_precision)
print(best_algorithm)
print(mean_recall)
print(mean_precision)

1:$10^3$
94.28104575163398
75.94620997972726
Bigrams_Random Forest
86.6470588235
84.8058127204


## No preprocessing Step

In [159]:
best_cost, max_recall, best_precision, best_algorithm, mean_recall, mean_precision = \
    summary(recall_avg_normalized, precision_avg_normalized, pattern=".*None.*(Bayes|SVM|.*Forest)")

Executing: None_Naive Bayes ...
Executing: None_Random Forest ...
Executing: None_SVM ...


In [160]:
print(best_cost)
print(max_recall)
print(best_precision)
print(best_algorithm)
print(mean_recall)
print(mean_precision)

1:25
100.0
44.00633773308192
None_Naive Bayes
92.2124183007
68.7617875467


## Bigrams

In [161]:
best_cost, max_recall, best_precision, best_algorithm, mean_recall, mean_precision = \
    summary(recall_avg_normalized, precision_avg_normalized, pattern=".*Bigra.*(Bayes|SVM|.*Forest)")

Executing: Bigrams_SVM ...
Executing: Bigrams_Random Forest ...
Executing: Bigrams_Naive Bayes ...


In [162]:
print(best_cost)
print(max_recall)
print(best_precision)
print(best_algorithm)
print(mean_recall)
print(mean_precision)

1:10
100.0
44.718557180116036
Bigrams_Naive Bayes
99.362745098
45.8460064913


## Combining bigrams and unigrams

In [163]:
best_cost, max_recall, best_precision, best_algorithm, mean_recall, mean_precision = \
    summary(recall_avg_normalized, precision_avg_normalized, pattern=".*Comb.*(Bayes|SVM|.*Forest)")

Executing: Combination_Naive Bayes ...
Executing: Combination_SVM ...
Executing: Combination_Random Forest ...


In [164]:
print(best_cost)
print(max_recall)
print(best_precision)
print(best_algorithm)
print(mean_recall)
print(mean_precision)

1:25
100.0
44.251912831181116
Combination_Naive Bayes
88.4705882353
81.6316299606


## Plot best results

In [23]:
plot_image(axis_costs, recall_avg_normalized, title="RECALL", ylim = [80., 100.5], 
           xlim = cxlim, models=pipes, name="recall", labels=labels, ylabel="Recall (%)", 
           loc='lower righ', markers="<Do|", pattern="(.*Bigram.*(Bayes|SVM|.*Forest)|None_Baseline)")

Executing: Bigrams_SVM ...
Executing: Bigrams_Naive Bayes ...
Executing: Bigrams_Random Forest ...


In [ ]:
plot_image(axis_costs, precision_avg_normalized, title="PRECISION", ylim = [0., 100], 
           xlim = cxlim, colors="rgbmyc", models=pipes, name="precision", labels = labels, ylabel="Precision (%)",
           loc ="lower right", markers="<Do|", pattern="(.*Bigram.*(Bayes|SVM|.*Forest)|None_Baseline)")